# Data Generation
## Graph Classes

we use the following graph classes:
- Chemical Graphs (Molecules)
- Random
- Small World
- Scale Free

## Data Save

We save the data as pickle file

In [ ]:
import numpy as np
import networkx as nx

def graph_load_batch(min_num_nodes = 20, max_num_nodes = 1000, name = 'ENZYMES', node_attributes = True, graph_labels=True):
    '''
    load many graphs, e.g. enzymes
    :return: a list of graphs
    '''
    print('Loading graph dataset: '+str(name))
    G = nx.Graph()
    # load data
    path = './../data/molecules/'+name+'/'
    data_adj = np.loadtxt(path+name+'_A.txt', delimiter=',').astype(int)
    if node_attributes:
        data_node_att = np.loadtxt(path+name+'_node_attributes.txt', delimiter=',')
    data_node_label = np.loadtxt(path+name+'_node_labels.txt', delimiter=',').astype(int)
    data_graph_indicator = np.loadtxt(path+name+'_graph_indicator.txt', delimiter=',').astype(int)
    if graph_labels:
        data_graph_labels = np.loadtxt(path+name+'_graph_labels.txt', delimiter=',').astype(int)


    data_tuple = list(map(tuple, data_adj))

    # add edges
    G.add_edges_from(data_tuple)
    # add node attributes
    for i in range(data_node_label.shape[0]):
        if node_attributes:
            G.add_node(i+1, feature = data_node_att[i])
        G.add_node(i+1, label = data_node_label[i])
    G.remove_nodes_from(list(nx.isolates(G)))

    # split into graphs
    graph_num = data_graph_indicator.max()
    node_list = np.arange(data_graph_indicator.shape[0])+1
    graphs = []
    max_nodes = 0
    for i in range(graph_num):
        # find the nodes for each graph
        nodes = node_list[data_graph_indicator==i+1]
        G_sub = G.subgraph(nodes)
        if graph_labels:
            G_sub.graph['label'] = data_graph_labels[i]
        if G_sub.number_of_nodes()>=min_num_nodes and G_sub.number_of_nodes()<=max_num_nodes:
            graphs.append(G_sub)
            if G_sub.number_of_nodes() > max_nodes:
                max_nodes = G_sub.number_of_nodes()
    print('Loaded')
    return graphs

In [1]:
import numpy as np
import networkx as nx
import os
import pickle
from tqdm.notebook import trange, tqdm

dataset = {
    "random": [],
    "smallworld": [],
    "scalefree": [],
}

def connect_all_components(G):
    complement = list(nx.k_edge_augmentation(G, k=1, partial=True))
    if nx.is_frozen(G):
        G = nx.Graph(G)
    G.add_edges_from(complement)
    assert nx.is_connected(G)

# generate 1000 random graphs using networkx and save them in the dataset
print("Generating random, smallworld and scalefree graphs...")
for j in trange(25, desc="Generating a new set of graphs"):
    for i in trange(5, 100):

        # random graph
        G = nx.gnp_random_graph(i, 0.5)
        if not nx.is_connected(G):
            print(f"connecting {G}")
            connect_all_components(G)
        dataset["random"].append(G)

        # small world graph
        G = nx.connected_watts_strogatz_graph(i, round(i / 2), 0.1)
        if not nx.is_connected(G):
            print(f"connecting {G}")
            connect_all_components(G)
        dataset["smallworld"].append(G)

        # scale free graph
        G = nx.barabasi_albert_graph(i, 1)
        if not nx.is_connected(G):
            print(f"connecting {G}")
            connect_all_components(G)
        dataset["scalefree"].append(G)

print("Done")

# read the chemical graphs from the ./data/molecules using graph_load_batch
"""
mutag_graphs = graph_load_batch(name="MUTAG", node_attributes=False)
for G in mutag_graphs:
    if not nx.is_connected(G):
        connect_all_components(G)
dataset["chemical"].extend(mutag_graphs)
nci1_graphs = graph_load_batch(name="NCI1", node_attributes=False)
for G in nci1_graphs:
    if not nx.is_connected(G):
        connect_all_components(G)
dataset["chemical"].extend(nci1_graphs)
nci109_graphs = graph_load_batch(name="NCI109", node_attributes=False)
for G in nci109_graphs:
    if not nx.is_connected(G):
        connect_all_components(G)
dataset["chemical"].extend(nci109_graphs)
"""

for key in dataset:
    for (i, G) in tqdm(enumerate(dataset[key]), total=len(dataset[key]), desc=f"Setting Node Label as Attribute for {key}"):
        node_labels = np.arange(G.number_of_nodes())
        degrees = nx.degree_centrality(G)
        betweenness = nx.betweenness_centrality(G)
        attrs = dict(zip(node_labels, node_labels))
        nx.set_node_attributes(G, attrs, "label")
        nx.set_node_attributes(G, degrees, "degree")
        nx.set_node_attributes(G, betweenness, "betweenness")

# save the dataset
import pickle
with open("dataset.pickle", "wb") as f:
    pickle.dump(dataset, f)


Generating random, smallworld and scalefree graphs...


Generating a new set of graphs:   0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

connecting Graph with 5 nodes and 4 edges


  0%|          | 0/95 [00:00<?, ?it/s]

...

  0%|          | 0/95 [00:00<?, ?it/s]

connecting Graph with 6 nodes and 5 edges
Done


Setting Node Label as Attribute:   0%|          | 0/2375 [00:00<?, ?it/s]

Setting Node Label as Attribute:   0%|          | 0/2375 [00:00<?, ?it/s]

Setting Node Label as Attribute:   0%|          | 0/2375 [00:00<?, ?it/s]

In [ ]:
# display the first graph of each type in the dataset
import matplotlib.pyplot as plt
import networkx as nx

# random graph
df = dataset["random"][0]
G = nx.from_pandas_edgelist(df)
nx.draw(G, with_labels=True)
plt.show()

# small world graph
df = dataset["smallworld"][0]
G = nx.from_pandas_edgelist(df)
nx.draw(G, with_labels=True)
plt.show()

# scale free graph
df = dataset["scalefree"][0]
G = nx.from_pandas_edgelist(df)
nx.draw(G, with_labels=True)
plt.show()

# chemical graph
df = dataset["chemical"][0]
G = nx.from_pandas_edgelist(df)
nx.draw(G, with_labels=True)
plt.show()